# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [ ]:
# import required libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql


Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [ ]:
# your code here

# import data set from Ironhack's database

##CONNECTION
driver = 'mysql+pymysql'
ip = '34.65.10.136'
username = 'data-students'
password = 'iR0nH@cK-D4T4B4S3'
db = 'orders'
connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'
engine = create_engine(connection_string)



#######################  WHICH TABLES WE WANT TO CHOOSE    #######################


query = 'SELECT * from orders'
orders = pd.read_sql(query,engine)
orders.head()

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

**Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?**


In [ ]:
customers = orders.groupby(orders.CustomerID).sum()
customers.head()

**Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?**

In [ ]:
per_VIP = customers.amount_spent.quantile(0.95)
print(per_VIP)

per_preferred= customers.amount_spent.quantile(0.75)
print(per_preferred)

**Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?**

*VIP CUSTOMERS*

In [ ]:
VIP = customers.loc[customers.amount_spent > per_VIP].sort_values('amount_spent', ascending = False)
print(VIP.shape)##number of VIP customer
VIP.head()


*PREFERRED CUSTOMERS*

In [ ]:
preferred = customers.loc[(customers.amount_spent > per_preferred) & (customers.amount_spent < per_VIP)].sort_values('amount_spent', ascending = False)
print(preferred.shape) ##number of preferred customer
preferred.head()


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

Merge two tables with the information requested.

In [ ]:
count_VIP = VIP[['amount_spent']]  #modify the VIP customers table to show just the amount_spent  (customerID is the index)

count_COUNTRY = orders[['Country','CustomerID']]      #modify the main orders table to show just the country and customerID .





1. make ther merge


2. group by the Country


3. sort by #CustomerID

In [ ]:
country_VIP = count_COUNTRY.merge(count_VIP,how = 'inner', on = 'CustomerID')
country_VIP = country_VIP.groupby(country_VIP.Country).sum()

country_VIP = country_VIP.sort_values('CustomerID',ascending =  False)

country_VIP.head()

**the count with more VIP customers are from UK**

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

Merge the three tables with the information requested:

    1.VIP table
    
    2.Preferred table
    
    3.Orders with customersID and Country

In [ ]:
count_VIP = VIP[['amount_spent']]  #modify the VIP customers table to show just the amount_spent  (customerID is the index)

count_COUNTRY = orders[['Country','CustomerID']]      #modify the main orders table to show just the country and customerID .

count_preferred = preferred[['amount_spent']]     #modify the preferred customers table to show just the amount_spent(customerID is the index)


count_preferred.head()

1. make ther merge Country_VIP (from the Q2) with the preferred one.



2. sort by #CustomerID

In [ ]:
count_preferred = count_preferred.reset_index()

In [ ]:
country_VIP = count_COUNTRY.merge(count_VIP, how = 'inner', on = 'CustomerID')
country_preferred = count_COUNTRY.merge(count_preferred, how = 'inner', on = 'CustomerID')
#country_VIP_preferred = country_VIP.join(count_preferred ,how='outer', on='CustomerID', lsuffix='_left', rsuffix='_right')

country_VIP_preferred = pd.concat([country_VIP, country_preferred])

country_VIP_preferred = country_VIP_preferred.groupby(country_VIP_preferred.Country).sum().sort_values('CustomerID', ascending = False)


country_VIP_preferred.head()





**the count with more VIP+ preferred customers are from UK**